In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [7]:
class JordanNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        super(JordanNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size) * torch.sqrt(torch.tensor(2.0 / (input_size + hidden_size))))
        self.Wh = nn.Parameter(torch.randn(hidden_size, output_size) * torch.sqrt(torch.tensor(2.0 / (hidden_size + output_size))))
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size) * torch.sqrt(torch.tensor(2.0 / (hidden_size + output_size))))
        self.bh = nn.Parameter(torch.zeros(hidden_size, 1))
        self.by = nn.Parameter(torch.zeros(output_size, 1))

        self.learning_rate = learning_rate
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.mse_loss = nn.MSELoss()
        
    def forward(self, x):
        y_t_1 = torch.zeros(self.output_size, 1)
        self.y_pred = []

        for t in range(len(x)):
            x_t = x[t].view(-1, 1)
            h_t = self.tanh(self.Wx @ x_t + self.Wh @ y_t_1 + self.bh)
            y_t = self.sigmoid(self.Wy @ h_t + self.by)
            self.y_pred.append(y_t)
            y_t_1 = y_t
        return self.y_pred

    def print_mse(self, X, y):
        y_pred = torch.cat(self.forward(X)).view(-1)
        mse = self.mse_loss(y.view(-1), y_pred)
        print(f"MSE: {mse.item()}")

    def backward(self, x, y):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        optimizer.zero_grad()

        y_pred = self.forward(x)
        y_pred_tensor = torch.cat(y_pred).view(-1)
        loss = self.mse_loss(y.view(-1), y_pred_tensor)
        loss.backward()
        optimizer.step()

In [8]:
# Parameter split_percent defines the ratio of training examples
def get_train_test(url, split_percent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data
 
sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [9]:
input_size, hidden_size, output_size = 1, 5, 1
self = JordanNetwork(input_size, hidden_size, output_size)

In [10]:
x = torch.tensor(train_data[:-1])  # Input data
y = torch.tensor(train_data[1:])  # Target data

In [11]:
for _ in range(10):
    self.forward(x)
    self.backward(x, y)
    self.print_mse(x,y)

MSE: 0.12345875054597855
MSE: 0.1143515333533287
MSE: 0.10619361698627472
MSE: 0.09888488054275513
MSE: 0.09233230352401733
MSE: 0.08645112067461014
MSE: 0.08116500079631805
MSE: 0.07640592753887177
MSE: 0.07211373001337051
MSE: 0.06823528558015823
